##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [ ]:
# Apache 라이선스, 버전 2.0("라이선스")에 따라 라이선스가 부여됩니다.
# 이 파일은 라이선스를 준수하는 경우에만 사용할 수 있습니다.
# 라이선스 전문은 다음에서 확인할 수 있습니다:
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# 관련 법률에 따라 요구되지 않는 한 또는 서면 동의가 없는 한,
# 이 소프트웨어는 "있는 그대로(AS IS)" 제공되며,
# 명시적이든 묵시적이든 어떠한 형태의 보증도 제공하지 않습니다.
# 사용 권한과 제한 사항에 대한 자세한 내용은 위의 라이선스를 참조하세요.

# MediaPipe Tasks를 활용한 자세 랜드마크 검출

이 노트북에서는 MediaPipe Tasks Python API를 사용하여 이미지에서 자세(포즈) 랜드마크를 검출하는 방법을 소개합니다.

## Preparation

Let's start with installing MediaPipe.


In [ ]:
!pip install -q mediapipe

그 다음, 기본 제공되는 모델 번들을 다운로드하세요.  
이 모델 번들에 대한 자세한 내용은 [MediaPipe 공식 문서](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker#models)를 참고하세요.

In [1]:
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

zsh:1: command not found: wget


## 🎨 Visualization Utilities

이 섹션은 **시각화 전용 도구 함수**를 정의합니다.

---

### 🔧 기능

- 👉 입력 이미지와 검출 결과(`detection_result`)를 받아  
- 👉 이미지 위에 사람의 **랜드마크와 관절 연결선**을 그림  

---

### 🧰 사용 기술

- **MediaPipe 시각화 유틸리티**  
  `solutions.drawing_utils.draw_landmarks(...)` 를 사용해 랜드마크를 이미지에 표시합니다.

---

### ⚠️ 참고

- 이 코드는 **아직 실행되는 부분은 없습니다.**
- 단순히 `draw_landmarks_on_image()`라는 **헬퍼 함수만 정의**합니다.
- 💡 즉, **분석은 하지 않고**, **그리기만 하는 도구**입니다.

이 함수는 여러 곳에서 재사용될 수 있도록 분리된 구조로 설계되어 있습니다.

In [ ]:
# 필요한 라이브러리 및 모듈을 가져옵니다.
from mediapipe import solutions  # MediaPipe 솔루션 (그리기 유틸리티 등 포함)
from mediapipe.framework.formats import landmark_pb2 # 랜드마크 데이터를 위한 프로토콜 버퍼 형식
import numpy as np # NumPy 라이브러리 (이미지 배열 처리에 사용)


def draw_landmarks_on_image(rgb_image, detection_result):
  """이미지 위에 감지된 포즈 랜드마크와 연결선을 그립니다.

  Args:
    rgb_image: 랜드마크를 그릴 원본 RGB 이미지 (NumPy 배열).
    detection_result: MediaPipe PoseLandmarker의 감지 결과 객체.

  Returns:
    랜드마크와 연결선이 그려진 이미지 (NumPy 배열).
  """
  # 감지 결과에서 포즈 랜드마크 목록을 가져옵니다.
  pose_landmarks_list = detection_result.pose_landmarks
  # 원본 이미지를 복사하여 주석(랜드마크)을 추가할 이미지를 만듭니다.
  annotated_image = np.copy(rgb_image)

  # 감지된 각 포즈(사람)에 대해 반복합니다.
  for idx in range(len(pose_landmarks_list)):
    # 현재 인덱스(idx)에 해당하는 포즈 랜드마크를 가져옵니다.
    pose_landmarks = pose_landmarks_list[idx]

    # 포즈 랜드마크를 그리기 위한 준비: landmark_pb2.NormalizedLandmarkList 형식으로 변환합니다.
    # draw_landmarks 함수는 이 형식을 입력으로 받습니다.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])

    # 이미지 위에 포즈 랜드마크와 연결선을 그립니다.
    solutions.drawing_utils.draw_landmarks(
      annotated_image, # 랜드마크를 그릴 이미지
      pose_landmarks_proto, # 그릴 랜드마크 데이터 (NormalizedLandmarkList 형식)
      solutions.pose.POSE_CONNECTIONS, # 랜드마크 간의 연결 정보 (어떤 랜드마크끼리 선으로 이을지 정의)
      solutions.drawing_styles.get_default_pose_landmarks_style()) # 랜드마크와 연결선의 기본 스타일 (색상, 두께 등)

  # 랜드마크와 연결선이 그려진 이미지를 반환합니다.
  return annotated_image


## Download test image

To demonstrate the Pose Landmarker API, you can download a sample image using the follow code. The image is from [Pixabay](https://pixabay.com/photos/girl-woman-fitness-beautiful-smile-4051811/).

In [ ]:
!wget -q -O image.jpg https://cdn.pixabay.com/photo/2019/03/12/20/39/girl-4051811_960_720.jpg

import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread("image.jpg")
cv2_imshow(img)

Optionally, you can upload your own image. If you want to do so, uncomment and run the cell below.

In [ ]:
# from google.colab import files
# uploaded = files.upload()

# for filename in uploaded:
#   content = uploaded[filename]
#   with open(filename, 'wb') as f:
#     f.write(content)

# if len(uploaded.keys()):
#   IMAGE_FILE = next(iter(uploaded))
#   print('Uploaded file:', IMAGE_FILE)

## 🧠 추론 실행 및 결과 시각화

이 단계는 **포즈 랜드마크 추론을 실제로 실행**하고  
결과를 **이미지 위에 시각화**하는 전체 파이프라인을 포함합니다.

---

### 🪜 전체 흐름

1. **모델 로드**  
   `.task` 모델 파일을 메모리에 불러옵니다.
   
2. **PoseLandmarker 옵션 설정**  
   모델 경로, 마스크 출력 여부 등 실행 조건을 정의합니다.
   
3. **이미지 로드**  
   분석할 이미지를 파일에서 불러옵니다.
   
4. **검출 수행**  
   `detector.detect(image)`를 통해 이미지에서 포즈를 인식합니다.
   
5. **검출 결과 시각화**  
   `draw_landmarks_on_image(...)`를 호출하여  
   이미지 위에 관절 위치와 연결선을 시각적으로 표시합니다.  
   👉 여기서 **이전에 정의한 시각화 함수가 사용됩니다.**

---

### 💡 핵심 함수 요약

| 함수 | 설명 |
|------|------|
| `detector.detect(image)` | 실제 자세 추정 수행 |
| `draw_landmarks_on_image(...)` | 검출 결과를 이미지 위에 시각화 (헬퍼 함수 사용) |

---

### 📖 참고 링크

더 다양한 설정 옵션을 알고 싶다면  
👉 [MediaPipe 공식 문서 보기](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python)

In [ ]:
# STEP 1: 필요한 모듈 가져오기 (Import the necessary modules.)
import mediapipe as mp  # MediaPipe 라이브러리를 mp라는 이름으로 가져옵니다.
from mediapipe.tasks import python # MediaPipe Tasks 관련 기본 기능을 가져옵니다.
from mediapipe.tasks.python import vision # MediaPipe의 비전 관련 작업(예: 포즈 감지) 기능을 가져옵니다.

# STEP 2: PoseLandmarker 객체 생성하기 (Create an PoseLandmarker object.)
# 모델 파일 경로 등 기본 옵션을 설정합니다.
base_options = python.BaseOptions(model_asset_path='Project/Climbing-Project/pose_landmarker_full.task')

# PoseLandmarker에 특화된 옵션을 설정합니다.
options = vision.PoseLandmarkerOptions(
    base_options=base_options, # 위에서 설정한 기본 옵션을 사용합니다.  / 모델 파일 경로 등
    num_poses=2, # 감지할 포즈(사람)의 최대 개수입니다. (최대 2명)
    min_detection_confidence=0.5, # 감지 신뢰도 임계값입니다. (0.0 ~ 1.0)
    min_tracking_confidence=0.5, # 추적 신뢰도 임계값입니다. (0.0 ~ 1.0)
    # 포즈 랜드마크의 세부 정보입니다.
    # pose_landmarks_to_return=vision.PoseLandmarkerOptions.PoseLandmarksToReturn.ALL, # 모든 랜드마크를 반환합니다.
    pose_landmarks_to_return=vision.PoseLandmarkerOptions.PoseLandmarksToReturn.NOSE_ONLY, # 코 부위 랜드마크만 반환합니다.
    # 포즈 랜드마크의 세부 정보입니다.
    # output_world_landmarks=True, # 월드 좌표계 랜드마크를 출력합니다.
    output_world_landmarks=False, # 월드 좌표계 랜드마크를 출력하지 않습니다.
    # output_segmentation_masks=True, # 사람 영역을 분할하는 마스크를 출력합니다.
    output_segmentation_masks=True) # 사람 영역을 분할하는 마스크도 출력하도록 설정합니다.

# 설정된 옵션들을 사용하여 PoseLandmarker 감지기(detector) 객체를 생성합니다.
detector = vision.PoseLandmarker.create_from_options(options)

# STEP 3: 입력 이미지 불러오기 (Load the input image.)
# "image.jpg" 파일로부터 이미지를 로드하여 MediaPipe가 처리할 수 있는 Image 객체로 만듭니다.
image = mp.Image.create_from_file("image.jpg")

# STEP 4: 입력 이미지에서 포즈 랜드마크 감지하기 (Detect pose landmarks from the input image.)
# 생성된 감지기(detector)를 사용하여 로드한 이미지에서 포즈 랜드마크를 감지합니다.
detection_result = detector.detect(image)
# 감지 결과는 detection_result 객체에 저장됩니다.
# 이 객체는 감지된 포즈 랜드마크와 관련된 정보를 포함하고 있습니다.
# 감지된 포즈 랜드마크는 detection_result.pose_landmarks 속성으로 접근할 수 있습니다.
# detection_result.pose_landmarks는 감지된 포즈 랜드마크의 리스트입니다.
# 각 포즈 랜드마크는 x, y, z 좌표를 포함하고 있습니다.
# 감지된 포즈 랜드마크는 이미지의 크기와 비율에 따라 정규화된 좌표로 표현됩니다.
# 감지된 포즈 랜드마크의 개수와 각 랜드마크의 좌표를 출력합니다.
print(f"Detected {len(detection_result.pose_landmarks)} pose landmarks.")
for idx, pose_landmarks in enumerate(detection_result.pose_landmarks):
  print(f"Pose {idx}:")
  for landmark in pose_landmarks:
    print(f"  x: {landmark.x}, y: {landmark.y}, z: {landmark.z}")
# 감지된 포즈 랜드마크의 개수와 각 랜드마크의 좌표를 출력합니다.
# 감지된 포즈 랜드마크의 개수는 detection_result.pose_landmarks의 길이로 확인할 수 있습니다.


# STEP 5: 감지 결과 처리하기 (Process the detection result.) 여기서는 시각화를 합니다.
# 앞에서 정의한 draw_landmarks_on_image 함수를 사용하여
# 원본 이미지(NumPy 배열 형태로 변환) 위에 감지된 랜드마크를 그립니다.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
# 랜드마크가 그려진 이미지를 화면에 표시합니다.
# (OpenCV는 BGR 색상 순서를 사용하므로 RGB 이미지를 BGR로 변환하여 표시합니다.)
# cv2_imshow는 Colab 환경에서 이미지를 표시하는 함수일 수 있습니다.
cv2_imshow(cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))


Visualize the pose segmentation mask.

In [ ]:
segmentation_mask = detection_result.segmentation_masks[0].numpy_view()
visualized_mask = np.repeat(segmentation_mask[:, :, np.newaxis], 3, axis=2) * 255
cv2_imshow(visualized_mask)

## 비디오로 시작하기

In [ ]:
# STEP 1: 필요한 모듈 가져오기
import mediapipe as mp
import cv2  # OpenCV 라이브러리 추가
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import time # 타임스탬프 생성을 위해 추가

# (이전에 정의된 draw_landmarks_on_image 함수가 있다고 가정합니다)
# from mediapipe import solutions
# from mediapipe.framework.formats import landmark_pb2
# def draw_landmarks_on_image(rgb_image, detection_result): ... (이전 코드 참고)

# STEP 2: 비디오 처리를 위한 PoseLandmarker 객체 생성
# 모델 파일 경로 설정 (사용자 환경에 맞게 수정 필요)
model_path = '/Users/laxdin24/Downloads/pose_landmarker_heavy.task'

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = vision.PoseLandmarker
PoseLandmarkerOptions = vision.PoseLandmarkerOptions
VisionRunningMode = vision.RunningMode

# 비디오 모드로 옵션 설정
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.VIDEO,
    output_segmentation_masks=True) # 필요한 경우 세그멘테이션 마스크 출력 설정

# STEP 3: 비디오 파일 열기 및 PoseLandmarker 생성
video_path = "your_video.mp4" # 처리할 비디오 파일 경로로 수정하세요.
cap = cv2.VideoCapture(video_path)

# 비디오의 FPS(초당 프레임 수) 가져오기 (타임스탬프 계산에 사용될 수 있음)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_duration_ms = int(1000 / fps) # 각 프레임의 지속 시간 (밀리초)

# PoseLandmarker 객체를 with 문 안에서 생성하여 사용 후 자동 정리
with PoseLandmarker.create_from_options(options) as landmarker:
    frame_index = 0
    while cap.isOpened(): # 비디오가 열려있는 동안 반복
        # STEP 4: 비디오에서 프레임 읽기
        success, frame = cap.read()
        if not success:
            print("비디오의 끝에 도달했거나 파일을 읽는 데 실패했습니다.")
            break

        # STEP 5: 프레임 처리 및 랜드마크 감지
        # OpenCV는 기본적으로 BGR 형식으로 이미지를 읽으므로 RGB로 변환
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # MediaPipe Image 객체로 변환
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

        # 현재 프레임의 타임스탬프 계산 (밀리초 단위)
        timestamp_ms = frame_index * frame_duration_ms

        # 비디오 모드용 감지 함수 호출 (타임스탬프 필요)
        detection_result = landmarker.detect_for_video(mp_image, timestamp_ms)

        # STEP 6: 결과 시각화 (선택 사항)
        # 원본 프레임(BGR)을 복사하여 사용 (annotated_image는 RGB를 반환하므로)
        annotated_frame_bgr = np.copy(frame)
        if detection_result.pose_landmarks:
            # draw_landmarks_on_image 함수는 RGB 이미지를 입력으로 받음
            annotated_frame_rgb = draw_landmarks_on_image(rgb_frame, detection_result)
            # 화면 표시를 위해 다시 BGR로 변환
            annotated_frame_bgr = cv2.cvtColor(annotated_frame_rgb, cv2.COLOR_RGB2BGR)

        # 결과 프레임 화면에 표시
        cv2.imshow('Pose Landmarker - Video', annotated_frame_bgr)

        frame_index += 1

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# STEP 7: 자원 해제
cap.release() # 비디오 캡처 객체 해제
cv2.destroyAllWindows() # 모든 OpenCV 창 닫기

print("비디오 처리가 완료되었습니다.")
